# Problema di permutation flowshop

## Job scheduling con algoritmo genetico

Ci sono m macchine e n posti di lavoro. Ogni job contiene esattamente m operazioni. L'i-esima operazione del lavoro deve essere eseguita sulla i-esima macchina. Nessuna macchina può eseguire più di un'operazione contemporaneamente. Per ogni operazione di ogni lavoro, viene specificato il tempo di esecuzione.
Le operazioni all'interno di un lavoro devono essere eseguite nell'ordine specificato. La prima operazione viene eseguita sulla prima macchina, poi (quando la prima operazione è terminata) la seconda operazione sulla seconda macchina, e così via fino all'ennesima operazione. Tuttavia, i lavori possono essere eseguiti in qualsiasi ordine.
L'obiettivo è quello di completare tutti i job nel minore tempo possibile, ovvero minimizzare il makespan.

### Ipotesi iniziali
Si considera un problema dove ci sono m macchine indipendenti, scheduling non preemptive e tempo di rilascio nullo per tutti i job, quindi i job posso essere eseguiti in qualsiasi istante e senza interruzioni.
Il modello di scheduling può essere modellato nel modo seguente:
F_m || C_max

In [1]:
from functions import *
import numpy as np
import re
import time

population_number: int = 60
iterations_number: int = 35 # numero di generazioni
p_mutation: float = 0.05    # probabilità di mutazione

bench_data = []             # matrice tempi di esecuzione job-macchina

In [2]:
dictionary = {}
file = open("taillard_20x5.txt", "rt")
with file as f:
    for line in f:
       (key, val) = line.split(':')
       dictionary[key] = val    
#print(dictionary)
file.close()

In [3]:
n_jobs = int(dictionary['number of jobs'])
n_machines = int(dictionary['number of machines'])
best_upper_bound = int(dictionary['upper bound'])
#print(n_jobs)
#print(n_machines)
#print(best_upper_bound)

In [4]:

processing_times = re.findall('\[(.*?)\]', dictionary['processing times'])
#print(processing_times)

['54 83 15 71 77 36 53 38 27 87 76 91 14 29 12 77 32 87 68 94', '79 3 11 99 56 70 99 60 5 56 3 61 73 75 47 14 21 86 5 77', '16 89 49 15 89 45 60 23 57 64 7 1 63 41 63 47 26 75 77 40', '66 58 31 68 78 91 13 59 49 85 85 9 39 41 56 40 54 77 51 31', '58 56 20 85 53 35 53 41 69 13 86 72 8 49 47 87 58 18 68 28']


In [5]:
i = 1
for machine_times in processing_times:
    list_times = machine_times.split(' ')
    list_times = [int(x) for x in list_times]
    bench_data.append(list_times)
    print('machine' + str(i) + ': ')
    print(list_times)
    i = i + 1
#print('\n')
#print('bench_data: ')
#print(bench_data)

machine1: 
[54, 83, 15, 71, 77, 36, 53, 38, 27, 87, 76, 91, 14, 29, 12, 77, 32, 87, 68, 94]
machine2: 
[79, 3, 11, 99, 56, 70, 99, 60, 5, 56, 3, 61, 73, 75, 47, 14, 21, 86, 5, 77]
machine3: 
[16, 89, 49, 15, 89, 45, 60, 23, 57, 64, 7, 1, 63, 41, 63, 47, 26, 75, 77, 40]
machine4: 
[66, 58, 31, 68, 78, 91, 13, 59, 49, 85, 85, 9, 39, 41, 56, 40, 54, 77, 51, 31]
machine5: 
[58, 56, 20, 85, 53, 35, 53, 41, 69, 13, 86, 72, 8, 49, 47, 87, 58, 18, 68, 28]


In [6]:
processing_time = [] # tempi di esecuzione del job j sulla macchina i
for i in range(n_jobs):
    temp = []
    for j in range(n_machines):
        temp.append(bench_data[j][i])
    print('job' + str(i + 1) + ': ')
    print(temp)
    processing_time.append(temp)


#print(processing_time)

job1: 
[54, 79, 16, 66, 58]
job2: 
[83, 3, 89, 58, 56]
job3: 
[15, 11, 49, 31, 20]
job4: 
[71, 99, 15, 68, 85]
job5: 
[77, 56, 89, 78, 53]
job6: 
[36, 70, 45, 91, 35]
job7: 
[53, 99, 60, 13, 53]
job8: 
[38, 60, 23, 59, 41]
job9: 
[27, 5, 57, 49, 69]
job10: 
[87, 56, 64, 85, 13]
job11: 
[76, 3, 7, 85, 86]
job12: 
[91, 61, 1, 9, 72]
job13: 
[14, 73, 63, 39, 8]
job14: 
[29, 75, 41, 41, 49]
job15: 
[12, 47, 63, 56, 47]
job16: 
[77, 14, 47, 40, 87]
job17: 
[32, 21, 26, 54, 58]
job18: 
[87, 86, 75, 77, 18]
job19: 
[68, 5, 77, 51, 68]
job20: 
[94, 77, 40, 31, 28]


In [7]:
start_time = time.time()
# creazione popolazione
population = initialize_population(population_number, n_jobs)
#print(population)

for evaluation in range(iterations_number):
    print('---- creazione ' + str(evaluation) + ' generazione ----\n')
    parents = get_parents(population, processing_time, n_jobs, n_machines)
    childs = []
    # operatore di crossover per generare nuove sequenze di scheduling
    childs.append(crossover(parents))
   # print('\n')
   # print(childs)

    # operatore di mutazione
    mutated_childs = []
    for child in childs:
        r = np.random.rand()
        if r < p_mutation:
            mutated_child = mutation(child)
            mutated_childs.append(mutated_child)

    childs.extend(mutated_childs)
    if len(childs) > 0:
        update_population(population, childs, processing_time, n_jobs, n_machines)
    print('---- generazione ' + str(evaluation) + ' creata ----\n')


---- creazione 0 generazione ----

fitness cumulata per sequenza: 
(0.018762251470176423, [3, 2, 5, 12, 7, 18, 19, 14, 1, 8, 16, 17, 4, 6, 15, 0, 9, 13, 10, 11])
(0.0330439652758331, [8, 3, 11, 2, 14, 10, 15, 0, 7, 4, 19, 17, 16, 12, 6, 1, 18, 9, 13, 5])
(0.0547465695883506, [1, 18, 17, 4, 7, 0, 15, 16, 11, 14, 9, 8, 10, 5, 2, 19, 6, 3, 13, 12])
(0.0677681321758611, [2, 10, 19, 17, 13, 15, 14, 5, 4, 6, 3, 11, 9, 12, 18, 0, 16, 8, 1, 7])
(0.08162979557546905, [10, 15, 9, 5, 2, 11, 17, 8, 12, 18, 6, 16, 19, 13, 14, 0, 7, 3, 4, 1])
(0.09969196303556427, [14, 4, 17, 9, 7, 5, 13, 15, 16, 12, 3, 6, 0, 19, 10, 8, 1, 18, 2, 11])
(0.12237468496219546, [1, 12, 7, 6, 2, 15, 0, 17, 13, 14, 16, 3, 19, 11, 18, 9, 8, 4, 10, 5])
(0.14309717166059926, [5, 10, 4, 11, 12, 6, 15, 17, 8, 9, 19, 16, 1, 13, 14, 0, 3, 7, 2, 18])
(0.152198263791655, [10, 9, 17, 19, 3, 5, 15, 16, 11, 18, 14, 0, 8, 13, 7, 12, 6, 4, 2, 1])
(0.17306076729207506, [16, 3, 2, 6, 8, 12, 5, 19, 7, 1, 11, 13, 4, 9, 10, 15, 17, 18, 14, 0

In [8]:
makespans = []
for individual in population:
    ind_makespan = (calc_makespan(individual, processing_time, n_jobs, n_machines), individual)
    makespans.append(ind_makespan)
makespans.sort(key=lambda x: x[0])

#best_seq = list(map(int, makespans[0][1]))
#best_makespan = get_makespan(n_machines, best_seq, bench_data)
best_seq = makespans[0][1]
best_makespan = makespans[0][0]
end_time = time.time()

In [9]:
for ms in makespans:
    print(ms)

(1404, [1, 2, 3, 10, 12, 5, 15, 16, 11, 18, 14, 0, 8, 13, 7, 6, 17, 4, 19, 9])
(1412, [15, 3, 13, 1, 10, 12, 4, 14, 11, 9, 8, 5, 18, 16, 6, 0, 7, 17, 2, 19])
(1420, [14, 13, 3, 16, 7, 9, 18, 0, 10, 17, 1, 12, 5, 11, 2, 4, 6, 19, 8, 15])
(1424, [2, 3, 12, 15, 16, 1, 5, 10, 14, 11, 9, 0, 8, 18, 7, 13, 6, 17, 4, 19])
(1429, [14, 9, 4, 12, 16, 2, 6, 8, 19, 13, 11, 1, 10, 0, 7, 18, 5, 3, 17, 15])
(1433, [2, 4, 13, 12, 16, 1, 5, 10, 14, 11, 9, 0, 8, 18, 7, 17, 6, 19, 15, 3])
(1437, [2, 18, 7, 5, 11, 15, 19, 14, 16, 3, 0, 10, 9, 8, 17, 1, 13, 4, 12, 6])
(1438, [7, 1, 4, 14, 11, 5, 13, 15, 16, 12, 3, 6, 0, 19, 10, 9, 8, 18, 17, 2])
(1442, [6, 16, 17, 0, 14, 1, 2, 7, 12, 8, 11, 10, 13, 9, 4, 19, 15, 18, 5, 3])
(1442, [10, 3, 7, 0, 1, 12, 4, 14, 11, 9, 8, 5, 18, 16, 6, 17, 15, 19, 13, 2])
(1442, [4, 5, 7, 1, 6, 15, 19, 14, 16, 3, 0, 10, 9, 8, 17, 12, 11, 2, 18, 13])
(1442, [4, 7, 0, 11, 14, 16, 17, 8, 1, 9, 6, 5, 19, 15, 18, 10, 2, 3, 13, 12])
(1443, [2, 3, 16, 12, 4, 1, 5, 10, 14, 11, 9, 0, 8, 

In [10]:
print(best_makespan)

1404


In [11]:
print(best_seq)

[1, 2, 3, 10, 12, 5, 15, 16, 11, 18, 14, 0, 8, 13, 7, 6, 17, 4, 19, 9]


In [12]:
print("--- %s secondi ---" % (end_time - start_time))

--- 0.14862728118896484 secondi ---
